In [ ]:
!python -m pip install opencv-python

In [2]:
!python -m pip install psd-tools
print("Installed")

   ---------------------------------------- 0.0/543.9 kB ? eta -:--:--
   - ------------------------------------- 20.5/543.9 kB 682.7 kB/s eta 0:00:01
   ------ --------------------------------- 92.2/543.9 kB 1.1 MB/s eta 0:00:01
   ------------- -------------------------- 184.3/543.9 kB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 256.0/543.9 kB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 348.2/543.9 kB 1.6 MB/s eta 0:00:01
   --------------------------------- ------ 450.6/543.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 543.9/543.9 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/67.6 kB ? eta -:--:--
   ---------------------------------------- 67.6/67.6 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
    --------------------------------------- 0.1/7.0 MB 2.6 MB/s eta 0:00:03
   - -------------------------------------- 0.2/7.0 MB 2.1 MB/s eta 0:00:04
 


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
from psd_tools import PSDImage

In [12]:
!python -m pip install opencv-contrib-python
import cv2
try:
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    print("Success! Saliency module is ready.")
except AttributeError:
    print("Module still not found. Check your environment.")

  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl (45.3 MB)
Module still not found. Check your environment.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\ATISHAY SG\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
class AssetTransformer:
    def __init__(self, input_path):
        self.master = self._load_asset(input_path)
        self.height, self.width = self.master.shape[:2]

    def _load_asset(self, path):
        if path.endswith('.psd'):
            psd = PSDImage.open(path)
            return np.array(psd.composite())
        return cv2.imread(path)

    def get_saliency_map(self):
        """Identifies key visual elements (text, logos)."""
        saliency = cv2.saliency.StaticSaliencyFineGrained_create()
        success, saliency_map = saliency.computeSaliency(self.master)
        # Normalize to 0-255
        return (saliency_map * 255).astype("uint8")

    def detect_elements(self):
        """Returns bounding boxes for key objects."""
        saliency = self.get_saliency_map()
        _, thresh = cv2.threshold(saliency, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        bboxes = [cv2.boundingRect(c) for c in contours if cv2.contourArea(c) > 500]
        return bboxes

    def smart_resize(self, target_w, target_h):
        """
        Executes repositioning logic. 
        If aspect ratio change is extreme, it switches to element reflow.
        """
        aspect_ratio = target_w / target_h
        bboxes = self.detect_elements()
        
        # Logic for Logo_Landscape (Extreme horizontal)
        if aspect_ratio > 3:
            return self._reflow_horizontal(target_w, target_h, bboxes)
        
        # Standard Smart Crop based on saliency center
        return cv2.resize(self.master, (target_w, target_h), interpolation=cv2.INTER_LANCZOS4)

    def _reflow_horizontal(self, tw, th, bboxes):
        """Extracts elements and arranges them side-by-side."""
        canvas = np.zeros((th, tw, 3), dtype="uint8")
        # Fill canvas with average background color or texture from master
        avg_color = cv2.mean(self.master)[:3]
        canvas[:] = avg_color
        
        # Simple repositioning: Place largest detected object (usually logo/text) in center
        # This is a placeholder for a more complex layout engine
        return canvas

    def save_all(self, output_dir):
        specs = [
            ("Image_Landscape", 1200, 628, "jpeg"),
            ("Image_Square", 1200, 1200, "jpeg"),
            ("Image_Portrait", 960, 1200, "jpeg"),
            ("Logo_Landscape", 1200, 300, "png"),
            ("Logo_Square", 1200, 1200, "png"),
        ]
        
        for name, w, h, ext in specs:
            img = self.smart_resize(w, h)
            cv2.imwrite(f"{output_dir}/sample_master_{name}.{ext}", img)


In [8]:
# Usage
transformer = AssetTransformer('D:/Datanodes_Assignment/sample_640_426.psd')
transformer.save_all('D:/Datanodes_Assignment/output/secondary_assets/')

AttributeError: module 'cv2' has no attribute 'saliency'